In [2]:
import sys
sys.path.append("../classes")

In [5]:
import pickle
from sklearn.svm import SVC

from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA

import numpy as np, pandas as pd
from geno_classifier import *
from itertools import starmap

import GEOparse

## GDS3257

In [2]:
lung_cancer_dataset = GEOparse.get_GEO('GDS3257').table

04-Jan-2018 22:07:51 INFO GEOparse - File already exist: using local version.
04-Jan-2018 22:07:51 INFO GEOparse - Parsing ./GDS3257.soft.gz: 
04-Jan-2018 22:07:51 DEBUG GEOparse - DATABASE: Geo
04-Jan-2018 22:07:51 DEBUG GEOparse - DATASET: GDS3257
04-Jan-2018 22:07:51 DEBUG GEOparse - SUBSET: GDS3257_1
04-Jan-2018 22:07:51 DEBUG GEOparse - SUBSET: GDS3257_2
04-Jan-2018 22:07:51 DEBUG GEOparse - SUBSET: GDS3257_3
04-Jan-2018 22:07:51 DEBUG GEOparse - SUBSET: GDS3257_4
04-Jan-2018 22:07:51 DEBUG GEOparse - SUBSET: GDS3257_5
04-Jan-2018 22:07:51 DEBUG GEOparse - SUBSET: GDS3257_6
04-Jan-2018 22:07:51 DEBUG GEOparse - SUBSET: GDS3257_7
04-Jan-2018 22:07:51 DEBUG GEOparse - SUBSET: GDS3257_8
04-Jan-2018 22:07:51 DEBUG GEOparse - SUBSET: GDS3257_9
04-Jan-2018 22:07:51 DEBUG GEOparse - SUBSET: GDS3257_10
04-Jan-2018 22:07:51 DEBUG GEOparse - SUBSET: GDS3257_11
04-Jan-2018 22:07:51 DEBUG GEOparse - SUBSET: GDS3257_12
04-Jan-2018 22:07:51 DEBUG GEOparse - SUBSET: GDS3257_13
04-Jan-2018 22:07:

## sample types

#### unhealthy

In [3]:
unhealthy_raw = '\
#GSM254629 = Value for GSM254629: Lung Tumor_GT00022; src: Adenocarcinoma of the Lung\
#GSM254648 = Value for GSM254648: Lung Tumor_GT01036; src: Adenocarcinoma of the Lung\
#GSM254694 = Value for GSM254694: Lung Tumor_GT01130; src: Adenocarcinoma of the Lung\
#GSM254701 = Value for GSM254701: Lung Tumor_GT01182; src: Adenocarcinoma of the Lung\
#GSM254728 = Value for GSM254728: Lung Tumor_GT01425; src: Adenocarcinoma of the Lung\
#GSM254726 = Value for GSM254726: Lung Tumor_GT01421; src: Adenocarcinoma of the Lung\
#GSM254639 = Value for GSM254639: Lung Tumor_GT01011; src: Adenocarcinoma of the Lung\
#GSM254652 = Value for GSM254652: Lung Tumor_GT01040; src: Adenocarcinoma of the Lung\
#GSM254700 = Value for GSM254700: Lung Tumor_GT01175; src: Adenocarcinoma of the Lung\
#GSM254625 = Value for GSM254625: Lung Tumor_GT00006; src: Adenocarcinoma of the Lung\
#GSM254636 = Value for GSM254636: Lung Tumor_GT01001; src: Adenocarcinoma of the Lung\
#GSM254659 = Value for GSM254659: Lung Tumor_GT01061; src: Adenocarcinoma of the Lung\
#GSM254680 = Value for GSM254680: Lung Tumor_GT01119; src: Adenocarcinoma of the Lung\
#GSM254686 = Value for GSM254686: Lung Tumor_GT01124; src: Adenocarcinoma of the Lung\
#GSM254718 = Value for GSM254718: Lung Tumor_GT01247; src: Adenocarcinoma of the Lung\
#GSM254674 = Value for GSM254674: Lung Tumor_GT01105; src: Adenocarcinoma of the Lung\
#GSM254668 = Value for GSM254668: Lung Tumor_GT01099; src: Adenocarcinoma of the Lung\
#GSM254697 = Value for GSM254697: Lung Tumor_GT01148; src: Adenocarcinoma of the Lung\
#GSM254704 = Value for GSM254704: Lung Tumor_GT01194; src: Adenocarcinoma of the Lung\
#GSM254707 = Value for GSM254707: Lung Tumor_GT01232; src: Adenocarcinoma of the Lung\
#GSM254714 = Value for GSM254714: Lung Tumor_GT01243; src: Adenocarcinoma of the Lung\
#GSM254722 = Value for GSM254722: Lung Tumor_GT01279; src: Adenocarcinoma of the Lung\
#GSM254627 = Value for GSM254627: Lung Tumor_GT00007; src: Adenocarcinoma of the Lung\
#GSM254630 = Value for GSM254630: Lung Tumor_GT00042; src: Adenocarcinoma of the Lung\
#GSM254633 = Value for GSM254633: Lung Tumor_GT00146; src: Adenocarcinoma of the Lung\
#GSM254670 = Value for GSM254670: Lung Tumor_GT01100; src: Adenocarcinoma of the Lung\
#GSM254716 = Value for GSM254716: Lung Tumor_GT01246; src: Adenocarcinoma of the Lung\
#GSM254720 = Value for GSM254720: Lung Tumor_GT01274; src: Adenocarcinoma of the Lung\
#GSM254729 = Value for GSM254729: Lung Tumor_GT01445; src: Adenocarcinoma of the Lung\
#GSM254654 = Value for GSM254654: Lung Tumor_GT01049; src: Adenocarcinoma of the Lung\
#GSM254656 = Value for GSM254656: Lung Tumor_GT01052; src: Adenocarcinoma of the Lung\
#GSM254631 = Value for GSM254631: Lung Tumor_GT00059; src: Adenocarcinoma of the Lung\
#GSM254657 = Value for GSM254657: Lung Tumor_GT01054; src: Adenocarcinoma of the Lung\
#GSM254664 = Value for GSM254664: Lung Tumor_GT01089; src: Adenocarcinoma of the Lung\
#GSM254672 = Value for GSM254672: Lung Tumor_GT01103; src: Adenocarcinoma of the Lung\
#GSM254692 = Value for GSM254692: Lung Tumor_GT01129; src: Adenocarcinoma of the Lung\
#GSM254645 = Value for GSM254645: Lung Tumor_GT01025; src: Adenocarcinoma of the Lung\
#GSM254666 = Value for GSM254666: Lung Tumor_GT01097; src: Adenocarcinoma of the Lung\
#GSM254675 = Value for GSM254675: Lung Tumor_GT01107; src: Adenocarcinoma of the Lung\
#GSM254678 = Value for GSM254678: Lung Tumor_GT01117; src: Adenocarcinoma of the Lung\
#GSM254688 = Value for GSM254688: Lung Tumor_GT01127; src: Adenocarcinoma of the Lung\
#GSM254690 = Value for GSM254690: Lung Tumor_GT01128; src: Adenocarcinoma of the Lung\
#GSM254696 = Value for GSM254696: Lung Tumor_GT01135; src: Adenocarcinoma of the Lung\
#GSM254705 = Value for GSM254705: Lung Tumor_GT01222; src: Adenocarcinoma of the Lung\
#GSM254642 = Value for GSM254642: Lung Tumor_GT01019; src: Adenocarcinoma of the Lung\
#GSM254661 = Value for GSM254661: Lung Tumor_GT01077; src: Adenocarcinoma of the Lung\
#GSM254698 = Value for GSM254698: Lung Tumor_GT01151; src: Adenocarcinoma of the Lung\
#GSM254641 = Value for GSM254641: Lung Tumor_GT01017; src: Adenocarcinoma of the Lung\
#GSM254647 = Value for GSM254647: Lung Tumor_GT01034; src: Adenocarcinoma of the Lung\
#GSM254663 = Value for GSM254663: Lung Tumor_GT01087; src: Adenocarcinoma of the Lung\
#GSM254682 = Value for GSM254682: Lung Tumor_GT01120; src: Adenocarcinoma of the Lung\
#GSM254709 = Value for GSM254709: Lung Tumor_GT01233; src: Adenocarcinoma of the Lung\
#GSM254721 = Value for GSM254721: Lung Tumor_GT01276; src: Adenocarcinoma of the Lung\
#GSM254724 = Value for GSM254724: Lung Tumor_GT01411; src: Adenocarcinoma of the Lung\
#GSM254650 = Value for GSM254650: Lung Tumor_GT01038; src: Adenocarcinoma of the Lung\
#GSM254687 = Value for GSM254687: Lung Tumor_GT01126; src: Adenocarcinoma of the Lung\
#GSM254637 = Value for GSM254637: Lung Tumor_GT01003; src: Adenocarcinoma of the Lung\
#GSM254684 = Value for GSM254684: Lung Tumor_GT01121; src: Adenocarcinoma of the Lung'

#### healthy

In [4]:
healthy_raw = '\
#GSM254649 = Value for GSM254649: Normal Lung_GT01036; src: Normal Lung Tissue\
#GSM254660 = Value for GSM254660: Normal Lung_GT01064; src: Normal Lung Tissue\
#GSM254693 = Value for GSM254693: Normal Lung_GT01130; src: Normal Lung Tissue\
#GSM254695 = Value for GSM254695: Normal Lung_GT01131; src: Normal Lung Tissue\
#GSM254702 = Value for GSM254702: Normal Lung_GT01182; src: Normal Lung Tissue\
#GSM254643 = Value for GSM254643: Normal Lung_GT01024; src: Normal Lung Tissue\
#GSM254727 = Value for GSM254727: Normal Lung_GT01421; src: Normal Lung Tissue\
#GSM254640 = Value for GSM254640: Normal Lung_GT01011; src: Normal Lung Tissue\
#GSM254626 = Value for GSM254626: Normal Lung_GT00006; src: Normal Lung Tissue\
#GSM254635 = Value for GSM254635: Normal Lung_GT01001; src: Normal Lung Tissue\
#GSM254653 = Value for GSM254653: Normal Lung_GT01047; src: Normal Lung Tissue\
#GSM254658 = Value for GSM254658: Normal Lung_GT01061; src: Normal Lung Tissue\
#GSM254681 = Value for GSM254681: Normal Lung_GT01119; src: Normal Lung Tissue\
#GSM254719 = Value for GSM254719: Normal Lung_GT01247; src: Normal Lung Tissue\
#GSM254673 = Value for GSM254673: Normal Lung_GT01105; src: Normal Lung Tissue\
#GSM254655 = Value for GSM254655: Normal Lung_GT01051; src: Normal Lung Tissue\
#GSM254669 = Value for GSM254669: Normal Lung_GT01099; src: Normal Lung Tissue\
#GSM254699 = Value for GSM254699: Normal Lung_GT01156; src: Normal Lung Tissue\
#GSM254703 = Value for GSM254703: Normal Lung_GT01194; src: Normal Lung Tissue\
#GSM254708 = Value for GSM254708: Normal Lung_GT01232; src: Normal Lung Tissue\
#GSM254715 = Value for GSM254715: Normal Lung_GT01243; src: Normal Lung Tissue\
#GSM254628 = Value for GSM254628: Normal Lung_GT00007; src: Normal Lung Tissue\
#GSM254634 = Value for GSM254634: Normal Lung_GT00146; src: Normal Lung Tissue\
#GSM254646 = Value for GSM254646: Normal Lung_GT01033; src: Normal Lung Tissue\
#GSM254671 = Value for GSM254671: Normal Lung_GT01100; src: Normal Lung Tissue\
#GSM254711 = Value for GSM254711: Normal Lung_GT01237; src: Normal Lung Tissue\
#GSM254717 = Value for GSM254717: Normal Lung_GT01246; src: Normal Lung Tissue\
#GSM254723 = Value for GSM254723: Normal Lung_GT01283; src: Normal Lung Tissue\
#GSM254730 = Value for GSM254730: Normal Lung_GT01445; src: Normal Lung Tissue\
#GSM254731 = Value for GSM254731: Normal Lung_GT01479; src: Normal Lung Tissue\
#GSM254632 = Value for GSM254632: Normal Lung_GT00059; src: Normal Lung Tissue\
#GSM254662 = Value for GSM254662: Normal Lung_GT01085; src: Normal Lung Tissue\
#GSM254677 = Value for GSM254677: Normal Lung_GT01113; src: Normal Lung Tissue\
#GSM254665 = Value for GSM254665: Normal Lung_GT01095; src: Normal Lung Tissue\
#GSM254691 = Value for GSM254691: Normal Lung_GT01129; src: Normal Lung Tissue\
#GSM254644 = Value for GSM254644: Normal Lung_GT01025; src: Normal Lung Tissue\
#GSM254667 = Value for GSM254667: Normal Lung_GT01097; src: Normal Lung Tissue\
#GSM254676 = Value for GSM254676: Normal Lung_GT01107; src: Normal Lung Tissue\
#GSM254679 = Value for GSM254679: Normal Lung_GT01117; src: Normal Lung Tissue\
#GSM254689 = Value for GSM254689: Normal Lung_GT01128; src: Normal Lung Tissue\
#GSM254706 = Value for GSM254706: Normal Lung_GT01222; src: Normal Lung Tissue\
#GSM254712 = Value for GSM254712: Normal Lung_GT01239; src: Normal Lung Tissue\
#GSM254713 = Value for GSM254713: Normal Lung_GT01240; src: Normal Lung Tissue\
#GSM254683 = Value for GSM254683: Normal Lung_GT01120; src: Normal Lung Tissue\
#GSM254710 = Value for GSM254710: Normal Lung_GT01233; src: Normal Lung Tissue\
#GSM254725 = Value for GSM254725: Normal Lung_GT01417; src: Normal Lung Tissue\
#GSM254651 = Value for GSM254651: Normal Lung_GT01038; src: Normal Lung Tissue\
#GSM254638 = Value for GSM254638: Normal Lung_GT01003; src: Normal Lung Tissue\
#GSM254685 = Value for GSM254685: Normal Lung_GT01121; src: Normal Lung Tissue'

In [5]:
clean = lambda string: string.replace(':', '').split()
predicate = lambda word: word.startswith('GSM')

healthy = set(filter(predicate, clean(healthy_raw)))
unhealthy = set(filter(predicate, clean(unhealthy_raw)))

### number of samples

In [6]:
print(len(healthy), len(unhealthy))

49 58


In [7]:
healthy & unhealthy

set()

In [8]:
lung_cancer_dataset[lung_cancer_dataset.columns[:5]].head()

,ID_REF,IDENTIFIER,GSM254629,GSM254648,GSM254694
0,1007_s_at,MIR4640,10.98850,10.69190,10.89780
1,1053_at,RFC2,6.82603,6.90960,6.80295
2,117_at,HSPA6,7.77559,7.68374,7.88498
3,121_at,PAX8,9.85506,10.13210,9.84110
4,1255_g_at,GUCA1A,4.82396,4.98489,4.87689


In [9]:
labels_ = [(h, 'healthy') for h in healthy] + [(d, 'unhealthy') for d in unhealthy]

In [10]:
X_, y_ = parse_database('GDS3257', labels=dict(labels_), n_jobs=-1)

04-Jan-2018 22:08:12 INFO GEOparse - File already exist: using local version.
04-Jan-2018 22:08:12 INFO GEOparse - Parsing ./GDS3257.soft.gz: 
04-Jan-2018 22:08:12 DEBUG GEOparse - DATABASE: Geo
04-Jan-2018 22:08:12 DEBUG GEOparse - DATASET: GDS3257
04-Jan-2018 22:08:12 DEBUG GEOparse - SUBSET: GDS3257_1
04-Jan-2018 22:08:12 DEBUG GEOparse - SUBSET: GDS3257_2
04-Jan-2018 22:08:12 DEBUG GEOparse - SUBSET: GDS3257_3
04-Jan-2018 22:08:12 DEBUG GEOparse - SUBSET: GDS3257_4
04-Jan-2018 22:08:12 DEBUG GEOparse - SUBSET: GDS3257_5
04-Jan-2018 22:08:12 DEBUG GEOparse - SUBSET: GDS3257_6
04-Jan-2018 22:08:12 DEBUG GEOparse - SUBSET: GDS3257_7
04-Jan-2018 22:08:12 DEBUG GEOparse - SUBSET: GDS3257_8
04-Jan-2018 22:08:12 DEBUG GEOparse - SUBSET: GDS3257_9
04-Jan-2018 22:08:12 DEBUG GEOparse - SUBSET: GDS3257_10
04-Jan-2018 22:08:12 DEBUG GEOparse - SUBSET: GDS3257_11
04-Jan-2018 22:08:12 DEBUG GEOparse - SUBSET: GDS3257_12
04-Jan-2018 22:08:12 DEBUG GEOparse - SUBSET: GDS3257_13
04-Jan-2018 22:08:

GSM254629 added with length 12666
GSM254648 added with length 12666
GSM254694 added with length 12666
GSM254701 added with length 12666
GSM254728 added with length 12666
GSM254726 added with length 12666
GSM254639 added with length 12666
GSM254652 added with length 12666
GSM254700 added with length 12666
GSM254625 added with length 12666
GSM254636 added with length 12666
GSM254659 added with length 12666
GSM254680 added with length 12666
GSM254686 added with length 12666
GSM254718 added with length 12666
GSM254674 added with length 12666
GSM254668 added with length 12666
GSM254697 added with length 12666
GSM254704 added with length 12666
GSM254707 added with length 12666
GSM254714 added with length 12666
GSM254722 added with length 12666
GSM254627 added with length 12666
GSM254630 added with length 12666
GSM254633 added with length 12666
GSM254670 added with length 12666
GSM254716 added with length 12666
GSM254720 added with length 12666
GSM254729 added with length 12666
GSM254654 adde

### Flux variability analysis

In [ ]:
results = flux_variance_analysis(X_, y_)
pickle.dump(results, open("../results/lung_cancer_1.results", "wb"))

In [6]:
results = pickle.load(open('../results/lung_cancer_1.results','rb'))
labels = pickle.load(open('../datasets/lung_cancer_1_y','rb'))

X_diff, y = preprocess_results(results, labels, use_diff_score=True, use_pathways=True)

### nested-cross validation over best pipeline (using diff-values)

In [7]:
classifiers = [
    (SVC, {
        'C': np.geomspace(1e-6, 1e6, num=10),
    })
]

feature_selection = [
    (PCA, {
    'n_components': range(3, 81+1, 6)
  })
]

In [8]:
trials, stats = nested_cross_validation(X_diff, y, build_pipelines(feature_selection, classifiers))

Fitting 10 folds for each of 140 candidates, totalling 1400 fits


[Parallel(n_jobs=-1)]: Done 232 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1400 out of 1400 | elapsed:   12.4s finished


1 trial done
----------
Fitting 10 folds for each of 140 candidates, totalling 1400 fits


[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 1400 out of 1400 | elapsed:   12.4s finished


2 trial done
----------
Fitting 10 folds for each of 140 candidates, totalling 1400 fits


...

[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 688 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 1400 out of 1400 | elapsed:   12.9s finished


9 trial done
----------
Fitting 10 folds for each of 140 candidates, totalling 1400 fits


[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 1024 tasks      | elapsed:    8.9s


10 trial done
----------


[Parallel(n_jobs=-1)]: Done 1400 out of 1400 | elapsed:   12.0s finished


In [10]:
stats.describe()

,fit_time,score_time,test_accuracy,test_f1,test_precision,test_recall
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.006520,0.002288,0.827717,0.830682,0.884762,0.805333
std,0.001170,0.000064,0.016529,0.014782,0.019649,0.015571
min,0.005612,0.002202,0.802323,0.804992,0.856429,0.776667
25%,0.005631,0.002244,0.817626,0.822792,0.874167,0.795000
50%,0.005827,0.002272,0.827323,0.832343,0.885595,0.808333
75%,0.007400,0.002331,0.834116,0.837756,0.896369,0.815833
max,0.008617,0.002387,0.860707,0.858353,0.914762,0.830000
